# Simulating Orbital Mechanics

In [4]:
import numpy as np
import sympy as sp
import sympy.physics.mechanics as spm
import scipy.integrate as spi
import matplotlib.pyplot as plt

## 2D Two Body Problem

System has rotational symmetry. Degrees of freedom $r_1\ r_2\ \theta_1\ \theta_2$ and derivatives $\dot{r}_1\ \dot{r}_2\ \dot{\theta}_1\ \dot{\theta}_2$.

In [5]:
G, m1, m2 = sp.symbols('G m_1 m_2')
r1, r2, th1, th2 = spm.dynamicsymbols('r_1 r_2 \\theta_1 \\theta_2')
r1d, r2d, th1d, th2d = tuple( s.diff() for s in (r1, r2, th1, th2) )